In [1]:
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.1/108.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.0: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla P100-PCIE-16GB. Max memory: 15.888 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 6.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=52,
    use_rslora=False,
    loftq_config=None
)

Unsloth 2024.10.0 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
prompt = """
### Instruction:
Вы — опытный ментор по программированию, помогающий студентам учиться на своих ошибках. Ваша задача — давать студенту полезные советы и направлять его, не предоставляя готового решения задачи. Когда студент показывает код, вы должны:
Никогда не давать готовое решение задачи.
Указать на ошибку или недочет, если они есть.
Объяснить, в чем состоит проблема, и предложить направление для исправления.
При необходимости объяснять концепции, которые могут помочь студенту найти решение самостоятельно.
Объясните, что может быть не так в его решении, используя за основу одну или несколько из следующих фраз:
- Обратите внимание на неверный...
- ...необходимо проверить, что...
- Вы забыли поставить...
- Необходимо использовать...
- Вы некорректно...
- Проверьте написание...
- В данном случае не нужно...
- ...неверный синтаксис...
- Ваш код охватывает не все возможные случаи...
- Попробуйте дополнить...
- Вы использовали неверную...
- Ошибка при обращении к...
- Проверьте, что...
- Ваш код использует неверный...

### Input:
{}

### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for input_, output in zip(inputs, outputs):
        text = prompt.format(input_, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

In [5]:
import pandas as pd
import pickle

In [6]:
train_solutions = pd.read_excel('/kaggle/input/hse-code-hack/data_v0/for_teams/train/solutions.xlsx')
train_tasks = pd.read_excel('/kaggle/input/hse-code-hack/data_v0/for_teams/train/tasks.xlsx')
train_tests = pd.read_excel('/kaggle/input/hse-code-hack/data_v0/for_teams/train/tests.xlsx')

test_solutions = pd.read_excel('/kaggle/input/hse-code-hack/data_v0/for_teams/test/solutions.xlsx')
test_tasks = pd.read_excel('/kaggle/input/hse-code-hack/data_v0/for_teams/test/tasks.xlsx')
test_tests = pd.read_excel('/kaggle/input/hse-code-hack/data_v0/for_teams/test/tests.xlsx')

synth = pd.read_csv('/kaggle/input/hsesynth/train_generated.csv')

with open('/kaggle/input/newsynth/NEW_hints_synth_total.pkl', 'rb') as f:
    synth_hints = pickle.load(f)

In [7]:
def combine_data(solutions_, tasks_, tests_):
    solutions, tasks, tests = solutions_.copy(), tasks_.copy(), tests_.copy()
    
    grouped_tests = tests.groupby('task_id').apply(
        lambda x: "; ".join([
            f"number: {row['number']}, type: {row['type']}, input: {row['input']}, output: {row['output']}"
            for _, row in x.iterrows()
        ])
    ).reset_index()
    grouped_tests = grouped_tests.rename(columns={0: 'tests'})
    
    merged_df = pd.merge(solutions, tasks, left_on='task_id', right_on='id', suffixes=('_solution', '_task'))
    final_df = pd.merge(merged_df, grouped_tests, how='left', on='task_id')
    
    return final_df

In [8]:
train = combine_data(train_solutions, train_tasks, train_tests)
test = combine_data(test_solutions, test_tasks, test_tests)

/tmp/ipykernel_24/1588904424.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_tests = tests.groupby('task_id').apply(
/tmp/ipykernel_24/1588904424.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_tests = tests.groupby('task_id').apply(


In [9]:
train_text = train[['description', 'author_solution', 'student_solution', 'author_comment', 'tests']]
test_text = test[['description', 'author_solution', 'student_solution', 'tests']]

synth['author_comment'] = synth_hints
synth_text = synth[['description', 'author_solution', 'student_solution', 'author_comment', 'tests']]

train_text = pd.concat([train_text, synth_text], axis=0).sample(frac=1, random_state=52).reset_index(drop=True)

In [10]:
input_template = '''
Условие задачи: {description}

Эталонное решение: {author_solution}

Решение студента: {student_solution}
'''

output_template = '''
Подсказка: {author_comment}
'''

In [11]:
train_df = pd.DataFrame({
    'input': train_text.apply(lambda row: input_template.format(
        description=row['description'],
        author_solution=row['author_solution'],
        student_solution=row['student_solution']
    ), axis=1),
    'output': train_text.apply(lambda row: output_template.format(
        author_comment=row['author_comment']
    ), axis=1)
})

test_df = pd.DataFrame({
    'input': test_text.apply(lambda row: input_template.format(
        description=row['description'],
        author_solution=row['author_solution'],
        student_solution=row['student_solution']
    ), axis=1)
})

In [12]:
# from sklearn.model_selection import train_test_split

# train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=52)

In [13]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
# val_dataset = Dataset.from_pandas(val_df)

train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
# val_dataset = val_dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/1455 [00:00<?, ? examples/s]

In [14]:
# from transformers import AutoModel, AutoTokenizer
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np

# bert_model_name = "DeepPavlov/rubert-base-cased-sentence"
# bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
# bert_model = AutoModel.from_pretrained(bert_model_name)

In [15]:
# def get_bert_embeddings(sentence):
#     inputs = bert_tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=256)
#     with torch.no_grad():
#         outputs = bert_model(**inputs)
#         embedding = outputs.last_hidden_state[:, 0, :].squeeze()
#     return embedding

In [16]:
# def compute_cossim(preds, labels):
#     pred_embeddings = torch.stack([get_bert_embeddings(pred) for pred in preds])
#     label_embeddings = torch.stack([get_bert_embeddings(label) for label in labels])
    
#     pred_embeddings_np = pred_embeddings.squeeze().numpy()
#     label_embeddings_np = label_embeddings.squeeze().numpy()
    
#     cossim_scores = cosine_similarity(pred_embeddings_np, label_embeddings_np)
#     mean_cossim = np.mean(np.diag(cossim_scores))
    
#     return mean_cossim

In [17]:
# def compute_metrics(eval_preds):
#     preds, labels = eval_preds

#     decoded_preds = [tokenizer.decode(pred, skip_special_tokens=True) for pred in preds]
#     decoded_labels = [tokenizer.decode(label, skip_special_tokens=True) for label in labels]
    
#     cossim_metric = compute_cossim(decoded_preds, decoded_labels)
    
#     return {"cossim": cossim_metric}

In [18]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

args = TrainingArguments(
    report_to='none',
    num_train_epochs=1,
    per_device_train_batch_size=2,
#     per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    learning_rate=2e-4,
    fp16=(not is_bfloat16_supported()),
    bf16=(is_bfloat16_supported()),
#     eval_strategy="steps",
    save_strategy="steps",
    save_steps=20,
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    seed=52,
    output_dir="outputs"
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
#     eval_dataset=val_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=args,
#     compute_metrics=compute_metrics
)

Map (num_proc=2):   0%|          | 0/1455 [00:00<?, ? examples/s]

In [19]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,455 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 182
 "-____-"     Number of trainable parameters = 83,886,080


Step,Training Loss
1,1.556100
2,1.545800
3,1.550700
4,1.465200
5,1.445400
6,1.141700
7,1.225000
8,0.937500
9,0.878800
10,0.766400
